# Convert 3DSSG's structure to GraphViz structure

## Load Files

In [3]:
import json

In [119]:
with open("objects.json", "r") as ro:
    data_obj = json.load(ro)
with open("relationships.json", "r") as rr:
    data_rel = json.load(rr)

print(type(data_obj))
obj_pretty = json.dumps(data_obj, indent=4)
rel_pretty = json.dumps(data_rel, indent=4)
print(len(data_obj['scans'][0]['objects']))

<class 'dict'>
40


## Exploring Data

In [120]:
# show keys (with hierarchy)
print('============================================================')
print('For "objects.json"')
print('============================================================')
print('data is constructed by: {keys}'.format(keys=data_obj.keys()))
print()
print('"scans" is constructed by: ', data_obj['scans'][0].keys())
print()
print('"objects" is constructed by: ', data_obj['scans'][0]['objects'][0].keys())
print('"label" in "objects" example: ', data_obj['scans'][0]['objects'][0]['label'])
print()
print('"id(attribute와 relationship 관련 id - scene 내에서의 id)" example: ', data_obj['scans'][0]['objects'][0]['id']) # instance id
print('"gloal_id(label에 따른 id)" example: ', data_obj['scans'][0]['objects'][0]['global_id']) # global instance id (among classes)
print('"attributes" example: ', data_obj['scans'][0]['objects'][0]['attributes'])
print('\n\n')
print('============================================================')
print('For "relationships.json"')
print('============================================================')
print('data is constructed by: ', data_rel.keys())
print()
print('"scans" is constructed by: ', data_rel['scans'][0].keys())
print()
print('"relationships" example: ', data_rel['scans'][0]['relationships'][0]) # 0: object, 1: object, 2: relationships id, 3: semantic relationship 

For "objects.json"
data is constructed by: dict_keys(['scans'])

"scans" is constructed by:  dict_keys(['scan', 'objects'])

"objects" is constructed by:  dict_keys(['ply_color', 'nyu40', 'eigen13', 'label', 'rio27', 'affordances', 'id', 'global_id', 'attributes'])
"label" in "objects" example:  floor

"id(attribute와 relationship 관련 id)" example:  1
"gloal_id(label에 따른 id)" example:  188
"attributes" example:  {'state': ['clean'], 'shape': ['flat'], 'lexical': ['inside', 'lower', 'horizontal'], 'color': ['beige']}



For "relationships.json"
data is constructed by:  dict_keys(['scans'])

"scans" is constructed by:  dict_keys(['relationships', 'scan'])

"relationships" example:  [2, 1, 15, 'standing on']


In [122]:
print('num of data_obj: ', len(data_obj['scans']))
print('num of data_rel: ', len(data_rel['scans']))

count = 0
for i in range(len(data_obj['scans'])):
    for j in range(len(data_rel['scans'])):
        if data_obj['scans'][i]['scan'] == data_rel['scans'][j]['scan']:
            count += 1
print('num of scans: ', count)

num of data_obj:  1482
num of data_rel:  1335
num of scans:  1335


## Create New Dictionaries

* 일단은 attributes 중 state만 사용하여 visualization -> attributes 개수가 많이 줄어듦

In [133]:
# convert to the structure for vizualization
objects = []
attributes = []
relationships = []

# at first, for the first scan -> id 기준으로 수정하기!
scan = data_obj['scans'][0]
obj_vals = []

for i in range(len(scan['objects'])):
    # for objects
    if scan['objects'][i]['label'] not in obj_vals:
        obj_vals.append(scan['objects'][i]['label'])
        objects.append({
            "name" : scan['objects'][i]['label']
        })
    
    # for attributes
    for attribute_list in scan['objects'][i]['attributes'].values(): # dict
        for attribute in attribute_list:
            attributes.append({
                "attribute" : attribute,
                "object" : int(scan['objects'][i]['id']) - 1
            })

print('num of objects: ', len(objects))
print('num of attributes: ', len(attributes))

# for relationships
for i in range(len(data_rel['scans'])):
    scan_rel = data_rel['scans'][i]
    if scan_rel['scan'] == scan['scan']:
        for relationship in scan_rel['relationships']:
            relationships.append({
                "predicate" : relationship[3], 
                "subject" : int(relationship[0]) - 1, # uncertain
                "object" : int(relationship[1]) - 1 # uncertain
            })

print('num of relationships: ', len(relationships))

num of objects:  26
num of attributes:  36
num of relationships:  350


## Create New Json file

In [134]:
# at first, 1st scan only
scan = str(0)
new_dict = {
    "url" : "https://" + scan + "th.jpg",
    "objects" : objects,
    "attributes" : attributes,
    "relationships" : relationships
}

with open("/Users/jjuke/Desktop/RVI/Projects/GraphViz/3dssg_for_viz.json", "w") as f:
    json.dump(new_dict, f, indent=4)